# Code for finding conserved protein sequences in the Genome

# Intro to Regex

How to use regex: https://workshop.veupathdb.org/WGC_Hinxton/2019/pdfs/6_FungiDB_Regular%20Expressions_Introduction.pdf

Special Characters  
. Match any character.  
'+' Matches "one or more of the preceding characters".  
'*' Matches "any number of occurrences of the preceding character", including 0.  
? Matches "zero or one occurrences of the preceding character".  
[ ] Matches any character contained in the brackets.  
[^ ] Match any character except those in the brackets.  
{n} Matches when the preceding character, or character range, occurs exactly n times.  
{n,} Matches when the preceding character occurs at least n times.  
{n,m} Matches when the preceding character occurs at least n times, but no more than m times.

# Import your Genome Sequence

Resources: https://biopython.org/wiki/SeqRecord  

Download Genbank file of the genome from benchling or NCBI 
- ‘Convert non standard annotation types’ is unchecked  
- Export feature colors is checked

Locate to where you downloaded your genome file  

Left click, copy path  

Paste in where the dots are in (SeqIO.parse(r"........................", "genbank"))




In [ ]:
# import SeqIO module
from Bio import SeqIO

# return the number of sequences, the ID of the sequence, the length of the sequence, the amount of features by parsing
for index, record in enumerate(SeqIO.parse(r"C:\Users\lexav\Downloads\x-autotrophicus-gj10.gb", "genbank")):
    print(
        "index %i, ID = %s, ,length %i, with %i features"
        % (index, record.id, len(record.seq), len(record.features))
        
    )

In [ ]:
# information within the object 
print(record)

# Create lists with bottom and top strand coding regions (protein sequences)  

resource: https://biopython.org/wiki/Intergenic_regions

In [1]:
import sys
import os
import Bio
from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord

def get_features(genbank_path):
    # parse the genbank file and save it as an object seq_record
    seq_record = SeqIO.read(open(genbank_path), "genbank")
    #create a list of coding sequence locations on the forward sense (5' -> 3')
    cds_list_plus = []
    #create a list of coding sequence locations on the anti sense (3' -> 5')
    cds_list_minus = []
    #create a list of coding sequences on the sense strand
    sequence_list_plus = []
    sequence_list_minus = []
    #create a list of coding sequences on the antisense strand
    # loop over genome file, get coding sequence features of each strand

    # for each feature in the features category of seq_records
    for feature in seq_record.features:
        # if the feature is a CDS type
        if feature.type == "CDS":
            # save the start position as mystart
            mystart = feature.location.start
            # save the end position as myend
            myend = feature.location.end
            # if it is antisense, add the information (mystart, myend, antisense) to the antisense list
            if feature.location.strand == -1:
                cds_list_minus.append((mystart, myend, -1))
            # if it is sense, add the information (mystart, myend, sense) to the sense list
            elif feature.location.strand == 1:
                cds_list_plus.append((mystart, myend, 1))
            #if neither of the previous two options, write something? in sense strand
            else:
                sys.stderr.write(
                    "No strand indicated %d-%d. Assuming +\n" % (mystart, myend)
                )
                cds_list_plus.append((mystart, myend, 1))

    #pospair receives the start and end coordinates of a CDS as a 2-tuple
    #iterate over each start and end pair
    for i, pospair in enumerate (cds_list_plus):
        #unpacks the start and end numbers, BUT i do not know why i need to include it in order for the code to work...
        mystart, myend, _ = pospair
        # for each pair, identify the sequence at the location specified
        gene_seq = seq_record.seq[mystart:myend]
        strand_string = '+'
        # add that to a new list of sequences
        sequence_list_plus.append(gene_seq)

    for i, pospair in enumerate (cds_list_minus):
        mystart, myend, _ = pospair
        gene_seq = seq_record.seq[mystart:myend]
        #flipped = gene_seq[::-1]
        rev = gene_seq.reverse_complement()
        strand_string = '-'
        sequence_list_minus.append(rev)

    
    return cds_list_plus, cds_list_minus, sequence_list_plus, sequence_list_minus

    
cds_list_plus, cds_list_minus, sequence_list_plus, sequence_list_minus = get_features(r"C:\Users\lexav\Downloads\x-autotrophicus-gj10.gb")

print(sequence_list_minus)
print(sequence_list_plus)



[Seq('ATGAGGGCGCAAATCGGTCCCGGGCAGTCCGATACCGGCACGGCCAGAAGCGGG...CAG'), Seq('ATGTCTACGACTGACGCCACCAGCCAGCCCAAGAAGCCGCCGGCCGAGGCCGGC...TGA'), Seq('GTGGTCCGGTTCGAGAACGTCGGCCTCAGGTACGGCATGGGTCCGGAGGTGCTG...TGA'), Seq('GTGATCCGTTCGCTTGCCGCCGCATCGGCCCTTGCGTTGCTTTTTTCGGCCGCC...TGA'), Seq('ATGAGCGAGGCCAGCGTCGCAAAGAGACCCGCCGCGCCGGTGCGCCATCTCGGC...TGA'), Seq('ATGAGCGATGGCGTGCAGGCTGTGGCTCTGGAGATCGAGGCGGTGAGCCACGCC...TGA'), Seq('ATGACCCCGACCGCCCCGCGCCGGCTCGTTGCCCTTCTCGCCGGAGCCGCTGCG...TGA'), Seq('ATGAGCCTTGCCTGGCGTCTCGAATGGCCGCGGCTGCGTCGGCCCCGTTTGCTT...TGA'), Seq('GTGCTGTTCGCGCTTCTTGCGCTGGTGGTGCTGGGCGGCATCGCCTTCGCGGTG...TGA'), Seq('GTGGCCAATGTGGTCGTGGTCGGCTCTCAGTGGGGCGACGAAGGCAAGGGCAAG...TAA'), Seq('ATGAATCACCTGAAGACGGCGATCTTGCTCGCCGGCATGACCGCCCTGTTCATG...TGA'), Seq('ATGTTCATTGTTCTTTATGCCTGCCTCATCAGTGCTCCGGCCACCTGCCGGGAG...TGA'), Seq('GTGGTAGCGCTCAACGGCGCCGCCGCCCCGCCCAAGCTCGTCGCGCGCATTCCC...TGA'), Seq('ATGGCGATCGTGCGCAGCGGTTCGGCGGAATGGAGCGGCGGCATCAAGGACGGG...TGA'), Seq('ATGCGCATCAACCTGGATGCGGACCAGC

### Now you have two lists, one list contains all of the top strands of proteins within the genome. The other contains all of the bottom strands of proteins within the genome.    

### We will now transcribe into mRNA

resource: https://biopython.org/docs/dev/Tutorial/chapter_seq_objects.html#transcription

In [2]:
# for the sense strand: 

def transcription(sensestrand):
    #create a list to append template DNA to
    template = []
    #create a list to append mRNA to
    mRNA = []
    # for each sequence in the sense strand list
    for seq in sensestrand:
        #take the reverse complement (this step is extra and goes no where, feel free to ignore)
        template_seq = seq.reverse_complement()
        # transcribe the sequence into mRNA
        mRNA_seq = seq.transcribe()
        #
        template.append(template_seq)
        mRNA.append(mRNA_seq)

    return mRNA

mRNAplus = transcription(sequence_list_plus)
mRNAminus = transcription(sequence_list_minus)

### The mRNA can be translated into proteins

In [3]:
def translation(mRNA_sequences):
    #create a list to append protein sequences to
    proteins = []
    # for each sequence in the sense strand list
    for seq in mRNA_sequences:
        #take the reverse complement (this step is extra and goes no where, feel free to ignore)
        protein_sequences = seq.translate()
        proteins.append(protein_sequences)

    return proteins

proteins_plus = translation(mRNAplus)
proteins_minus = translation(mRNAminus)

print(proteins_plus)
print(type(proteins_plus))

[Seq('MNLPNVSVLYDENRIAARNHALAVEIAAFAPERLLAVAVLKGSFIFAADLLRAM...LE*'), Seq('MMSGARFVFALLFAGATFFGVTGRSLAAPVPVAIFGFELFDDTADQRKSIQAEQ...SQ*'), Seq('MRQMIAAAAALAVAMIPLAADAHGPTRKKVTESVEINAPADKVWAVISNFQDMS...GS*'), Seq('VRDGLDGALAIAGLIGPDAPDDLSRAARALSRAPHAVALDLASALAENLPVHRR...DS*'), Seq('MKTPRSAPASSRRPGGSPKPVAATPGLSARAVAVDALEAVLRQGAALEEALDGP...GG*'), Seq('MGIWNGSAGPLQAVARNLVGTLAGKGHAHTPGGAVSRGGLRERTQLATLLVELW...RG*'), Seq('VAKISKVLVGEALVGDGNEVAHIDLIIGPRGSAAETAFVTSLTNNKDGFTSLLA...AA*'), Seq('LPGWFVMKASAAPEGRDATRXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...HS*'), Seq('MAQLSILDLVRVREGFGPRQALDNARDLAAHAEDWGYRRFWVAEHHNMQGIASA...AV*'), Seq('MSFCPGQQDAGSQPLPARSRFALALAPTLSARQRARLKPWLVALAVAGMAAGLV...CR*'), Seq('VRLVCLPHAGGSAVLYRSWKAALAGLATVDSPELPGRGSRHGAPVPASLAALAE...RA*'), Seq('VPDLSSPVDAARLAPAASLLAIARVADAPVNPDLSIALPAGEAARIARHVKADD...RM*'), Seq('MKLIVERLVVESLVDAFPDQAHLLAQPRLSSRMELGLDALKPDELAHLHELVLR...LA*'), Seq('MTPARIGIVTISDRASSGTYEDLSGPAIEAWLRRAITSPWVADYRVIPDGFESV...GA*'), Seq('MPILKEFKEFAVRGNVVDLAVGVIIGAA

### Now you have two lists with all potential annotated proteins within the genome. We will now create a function that searches each one of these proteins for regex sequences

this code looks for conserved sequences of the group one hydrogenase

In [4]:
import re

#function for detecting regex signatures
def signatures(sequence_list, signature):
    # for each protein in the list of proteins
    for sequence in sequence_list:
        #find a match to the given protein sequence
        match = re.search(signature, str(sequence))
        #if there is a match, print the phrase 'motif found' and the protein sequence that is the match
        if match:
            print("motif found:", match.group(0))
            found = True


# write each regex, Hydrogenase signatures: NiFe

# group one are membrane bound H2 uptake hydrogenases
L1_groupone = r"[EGMQS]R?.C?[GR][IV]C.{3}[HT].{3}[AGS].{0,4}[VANQD]"
L2_groupone = r"[AFGIKLMV][HMR]..[HR][AS][AFLY][DN]PC[FILMV].C[AGS].H"


### Using the function to check your genome for signatures  

- signatures(sequence_list, signature)

    - signatures is the title of the function
    - sequence_list is the list of protein sequences you are going to search through
    - signature is the regex you are looking for (we just defined the objects in the previous code blocks)
    - see code block below for examples

In [5]:
print('group one L1:')
signatures(proteins_plus, L1_groupone)
signatures(proteins_minus, L1_groupone)

group one L1:
motif found: GDRVCGYQTYGAARAYVA
motif found: ERICGVCTGCHALTSVRAV
motif found: GLRVCAFGHMGDGNIHYN


### note: L1 is a conserved cysteine residue required to ligate H2-binding metal centers in this example

# Searching genome for multiple conserved sequences 

In [13]:

# define a function to input sequences and signatures
def multiple_signatures(sequence_list, signature, signature2, signature3):
    #create a list that the matches will go in
    match_list = []

    #for each protein in the list
    for sequence in sequence_list:
        
        #turn the protein sequence into a string
        sequence = str(sequence)

        # first signature, look for a signature that matches in the sequence

        match1 = re.search(signature, sequence)

        #if there is a match, look for another match in the list of protein sequences 
            #note: A good update to the code would be looking for a match within the sequence the match was found. 
        if match1:
            # second signature
            match2 = re.search(signature2, sequence)
            if match2:
                # third signature
                match3 = re.search(signature3, sequence)
                if match3:
                    # append all matches as strings
                    match_list.append([match1.group(0), match2.group(0), match3.group(0)])

    df = pd.DataFrame(match_list, columns=["Match1", "Match2", "Match3"])
    return df

#print title of conserved sequences
print('Group 2D1')
#Group 2D1 of hydrogenases
FAD_2D1 = r"IDGDTAQVGPG"
NADH_2D1 = r'DR.FAG.DTLAT'

multiple_signatures(proteins_plus, FAD_2D1, NADH_2D1, NADH_2D1)
multiple_signatures(proteins_minus, FAD_2D1, NADH_2D1, NADH_2D1)

Group 2D1


,Match1,Match2,Match3
0,IDGDTAQVGPG,DRFFAGSDTLAT,DRFFAGSDTLAT


### Looking for an amino acid at a specific position on a protein

In [14]:
def check_specific_residues(protein_list):
    """
    Check each protein sequence in the list for specific residues at given positions.
    
    Conditions:
    - D at position 249
    - N at position 123
    - T at position 156
    
    Prints the protein index and sequence if conditions are met.
    
    Parameters:
    protein_list (list of str): List of protein sequences
    """
    for idx, protein in enumerate(protein_list):
        sequence = str(protein)
        
        # Ensure protein is long enough for the positions we are checking
        if len(sequence) >= 249:
            if sequence[248] == 'D' and sequence[122] == 'N' and sequence[155] == 'T':
                print(f"Protein {idx} has D at 249, N at 123, and T at 156: {sequence}")


# Example usage
check_specific_residues(proteins_plus)
check_specific_residues(proteins_minus)
